In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra
import cassio
from dotenv import load_dotenv
load_dotenv()

In [ ]:
groq_api_key = os.environ.get("GROQ_API_KEY")
ASTRA_DB_APPLICATION_TOKEN="AstraCS:vPfILQUUvrhrHOCfNlosWSdr:fd4cbafe7df8c9af41e18f4bf81648b69da816909cc4b55a27337396ea77d671" # enter the "AstraCS:..." string found in in your Token JSON file"
ASTRA_DB_ID="53aed175-1165-4a25-846d-0bb703c24d70"
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [ ]:
text_documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs=text_splitter.split_documents(text_documents)
docs[:5]

In [ ]:
## Convert Data Into Vectors and store in AstraDB
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
embeddings=OpenAIEmbeddings()
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None

)

In [ ]:
llm=ChatGroq(groq_api_key=groq_api_key,
         model_name="mixtral-8x7b-32768")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>

Question: {input}""")

In [ ]:
astra_vector_index.query("Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique",llm=llm)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever=astra_vector_store.as_retriever()
document_chain=create_stuff_documents_chain(llm,prompt)
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique"})
response

In [ ]:
response["answer"]